# 神经风格迁移

* **风格**(style)是指图像中不同空间尺度的纹理\颜色和视觉图案

* **内容**(content)是指图像的高级宏观结构.

实现风格迁移背后的关键概念与所有深度学习算法的核心思想是一样的:
定义一个损失函数制定想要实现的目标,然后将这个损失最小化.

## 内容损失

内容损失的一个很好的候选者就是两个激活之间的L2范数,一个激活是预训练的卷积神经网络更靠顶部的某层在目标图像上计算得到的激活,另一个激活是同一层在生成图像上计算得到的激活.

这可以保证,在更靠近顶部的层看来,生成图像与原始目标图像看起来很相似.


## 风格损失

风格损失的目的是在风格参考图像与生成图像之间,在不同的层激活内保存想死的内部相互关系.这保证了在风格参考图像与生成图像之间,不同空间尺度找到的纹理看起来都很相似.


简而言之,可以使用预训练的卷积神经网络来定义一个具有以下特点的损失

* 在目标内容图像和生成图像之间保持相似的较高层激活,从而能够保留内容.卷积神经网络应该能够"看到"目标图像和生成的图像包含相同的内容

* 在较低层和较高层的激活中保持类似的相互关系(correlation),从而能够保留风格.特征相互关系捕捉到的是纹理(texture),生成图像和风格参考图像在不同的空间尺度上应该具有相同的纹理.

## 用Keras实现神经风格迁移 

使用预训练的VGG19

#### 定义初始变量

In [2]:
from keras.preprocessing.image import load_img, img_to_array

Using TensorFlow backend.


In [4]:
target_image_path = 'img/beauty.jpg'
style_reference_image_path = 'img/Starry_Night.jpg'

# 生成图像的尺寸
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

#### 辅助函数

In [5]:
import numpy as np
from keras.applications import vgg19

def preprocess_image(image_path):
    img = load_img(image_path, target_size = (img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    # 减去ImageNet的平均像素值
    # 使其中心为0,这里相当于vgg19.preprocess_input的逆操作
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 将图像由BGR格式转换为RGB格式
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x
    

#### 加载预训练的VGG19网络,并将其应用于3张图片

In [7]:
from keras import backend as K
target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1, img_height, img_width, 3))

input_tensor = K.concatenate([target_image, style_reference_image, combination_image], axis = 0)

model = vgg19.VGG19(input_tensor = input_tensor, 
                    weights = 'imagenet',
                    include_top = False)

print('Model loaded')

80142336/80134624 [==============================] - 675s 8us/step
Model loaded


#### 内容损失

In [8]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

#### 风格损失

In [9]:
def gram_matrix(x):
    featerus = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

#### 总变差损失

In [10]:
def total_variation_loss(x):
    a = K.square(
        x[:, :img_height - 1, :img_width - 1, :] -
        x[:, 1:, :img_width - 1, :])
    b = K.square(
        x[:, :img_height - 1, :img_width - 1, :] -
        x[:, img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))